In [55]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import time
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np
import cv2
from PIL import Image

data_dir='C:/Users/rajpal/Downloads/Module 22/faces/'

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

expression_map = {
            'neutral': 0,
            'happy': 1,
            'sad': 2,
            'angry': 3,
            'surprise': 4,
            'fear': 5,
            'disgust': 6
        }

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Initializes the FaceDataset class.

        Args:
            root_dir (str): The root directory containing the image files.
            transform (transforms.Compose, optional): The transformations to apply to the images.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        self.load_image_files()

    def load_image_files(self):
        """Loads all image files with a .pgm extension from the root directory and its subdirectories."""
        for root, _, files in os.walk(self.root_dir):
            for file in files:
                if file.endswith('.pgm'):
                    self.image_files.append(os.path.join(root, file))

    def __len__(self):
        """Returns the number of image files in the dataset."""
        return len(self.image_files)

    def __getitem__(self, idx):
        """Retrieves an image and its corresponding labels from the dataset.

        Args:
            idx (int): The index of the image to retrieve.

        Returns:
            tuple: A tuple containing the image, the 'is_mitchell' label, and the expression label.
        """
        img_path = self.image_files[idx]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (32, 32))

        img_name = os.path.basename(img_path)
        parts = img_name.split('_')

        is_mitchell = int(parts[0] == 'mitchell')
        expression = expression_map[parts[2]]

        if self.transform:
            image = self.transform(image)

        return image, is_mitchell, expression



def img_loader(filename):
    return cv2.resize(cv2.imread(filename, cv2.IMREAD_GRAYSCALE), (32,32))


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3_mitchell = nn.Linear(84, 1) 
        self.fc3_expression = nn.Linear(84, 7)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        is_mitchell = torch.sigmoid(self.fc3_mitchell(x))
        expression = self.fc3_expression(x)
        return is_mitchell, expression
    
def train(check_point_file, data_dir, *args, **kwargs):

    dataset = FaceDataset(data_dir, transform)
    train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
    model = Net()
    criterion_mitchell = nn.BCELoss()
    criterion_expression = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    num_epochs = 100
    image_count = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss_mitchell = 0.0
        running_loss_expression = 0.0
    
        for i, data in enumerate(train_loader, 0):
            inputs, labels_mitchell, labels_expression = data
            optimizer.zero_grad()
            outputs_mitchell, outputs_expression = model(inputs)
            loss_mitchell = criterion_mitchell(outputs_mitchell.squeeze(), labels_mitchell.float())
            loss_expression = criterion_expression(outputs_expression, labels_expression)
            loss = loss_mitchell + loss_expression
            loss.backward()
            optimizer.step()
            running_loss_mitchell += loss_mitchell.item()
            running_loss_expression += loss_expression.item()
    
        print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], '
              f'Loss Mitchell: {running_loss_mitchell / 100:.4f}, '
              f'Loss Expression: {running_loss_expression / 100:.4f}')

    print('Finished Training')
    torch.save(model, check_point_file)
    print('Saved the model')


def test(check_point_file, test_dir, *args, **kwargs):

    model = torch.load(check_point_file)
    model.eval()
    correct_mitchell = 0
    total_mitchell = 0
    correct_expression = 0
    total_expression = 0
    dataset = FaceDataset(root_dir=test_dir, transform=transform)
    test_loader = DataLoader(dataset, batch_size=64, shuffle=True)
    with torch.no_grad():
        for data in test_loader:
            images, labels_mitchell, labels_expression = data
            outputs_mitchell, outputs_expression = model(images)
            
            predicted_mitchell = (outputs_mitchell.squeeze() > 0.5).int()
            total_mitchell += labels_mitchell.size(0)
            correct_mitchell += (predicted_mitchell == labels_mitchell).sum().item()
            
            _, predicted_expression = torch.max(outputs_expression, 1)
            total_expression += labels_expression.size(0)
            correct_expression += (predicted_expression == labels_expression).sum().item()

    print(f'Accuracy of the network on Task 1 (Mitchell): {100 * correct_mitchell / total_mitchell:.2f}%')
    print(f'Accuracy of the network on Task 2 (Expression): {100 * correct_expression / total_expression:.2f}%')
    print('Finished Testing')


def main():
 
    parser = argparse.ArgumentParser()
    parser.add_argument('--data', type=str, default='./faces', metavar='N',
                        help='Path to directory containing faces dataset.')
    parser.add_argument('--predict', type=str, metavar='N',
                        help='Path to image for prediction.')
    
    args = parser.parse_args()
    data_dir = args.data
    checkpoint_dir = './check_point/fc_model.pt'

    if args.predict:
        print("Run the prediction")
        predict(checkpoint_dir, args.predict)
    else:
        print("Run model training and testing")
        train(checkpoint_dir, data_dir)
        test(checkpoint_dir, data_dir + '/test')



In [56]:
check_point_file = 'C:/Users/rajpal/Downloads/Module 22/check_point/fc_model.pt'
train(check_point_file, data_dir)

Epoch [1/100], Step [30/30], Loss Mitchell: 0.1928, Loss Expression: 0.5819
Epoch [2/100], Step [30/30], Loss Mitchell: 0.1601, Loss Expression: 0.5666
Epoch [3/100], Step [30/30], Loss Mitchell: 0.1047, Loss Expression: 0.5341
Epoch [4/100], Step [30/30], Loss Mitchell: 0.0560, Loss Expression: 0.4533
Epoch [5/100], Step [30/30], Loss Mitchell: 0.0644, Loss Expression: 0.4238
Epoch [6/100], Step [30/30], Loss Mitchell: 0.0547, Loss Expression: 0.4228
Epoch [7/100], Step [30/30], Loss Mitchell: 0.0566, Loss Expression: 0.4197
Epoch [8/100], Step [30/30], Loss Mitchell: 0.0559, Loss Expression: 0.4195
Epoch [9/100], Step [30/30], Loss Mitchell: 0.0591, Loss Expression: 0.4197
Epoch [10/100], Step [30/30], Loss Mitchell: 0.0575, Loss Expression: 0.4176
Epoch [11/100], Step [30/30], Loss Mitchell: 0.0558, Loss Expression: 0.4176
Epoch [12/100], Step [30/30], Loss Mitchell: 0.0558, Loss Expression: 0.4173
Epoch [13/100], Step [30/30], Loss Mitchell: 0.0557, Loss Expression: 0.4170
Epoch [1

In [57]:
test(check_point_file, data_dir)

C:\Users\rajpal\AppData\Local\Temp\ipykernel_8344\948452374.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(check_point_file)


Accuracy of the network on Task 1 (Mitchell): 95.51%
Accuracy of the network on Task 2 (Expression): 26.71%
Finished Testing
